In [1]:
import numpy as np
import tbmodels
import h5py

In [2]:
def apply_efield(h0,efield,z_pos):
    h = h0.copy()
    for i in np.arange(h0.shape[0]):
        h[i,i] = h0[i,i]+efield*z_pos[i]
    return h

def find_efermi_uniform_sampled_BZ(eigvals_all, nfilling, nk):
    nelec_tot = nfilling * nk
    eigvals_all_sorted = np.sort(eigvals_all.flatten())
    vbm = eigvals_all_sorted[nelec_tot-1]
    cbm = eigvals_all_sorted[nelec_tot]
    efermi = (vbm + cbm) / 2
    return vbm, cbm, efermi

In [3]:
# generate a list of uniformly sampled k points in the BZ
nk = 21
k_x = np.linspace(-0.5, 0.5, nk+1)[:-1]
k_y = np.linspace(-0.5, 0.5, nk+1)[:-1]
# make a 2D grid of k points
kpoints = np.array([[kx, ky, 0] for kx in k_x for ky in k_y])

In [4]:
nwann = 56
nfilling = 36

wpath = f"../tb_model/model_o{nwann}/"

model = tbmodels.Model.from_wannier_files(
            hr_file = f'{wpath}/wte2.o{nwann}_hr.dat',
            xyz_file = f'{wpath}/wte2.o{nwann}_centres.xyz',
            win_file = f'{wpath}/wte2.o{nwann}.win')

# apply on-site electric fields to the wannier orbitals
c = 6.576*4.0316*0.529177
z_pos = model.pos[:,2]*c

z_pos_sym = z_pos.copy()
for iz in np.arange(z_pos.shape[0]):
    z_pos_sym[iz] = 0.5*(z_pos[iz]+z_pos[iz+(-1)**(iz%2)])

In [7]:
# generate the band structure on a given k-path for the following efields
efields = [-0.5, -0.35, -0.3, -0.2,-0.15, -0.1, -0.05, 0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.35, 0.5]

with h5py.File("./data/bulk_uniform_sampled_BZ_efields.h5", 'w') as f:
    f.create_dataset("kpoints", data=kpoints)
    for efield in efields:
        h0_k = model.hamilton(kpoints)
        h_k = [apply_efield(h0,efield,z_pos) for h0 in h0_k]
        eigvals, eigvecs = np.linalg.eigh(h_k)
        vbm, cbm, efermi = find_efermi_uniform_sampled_BZ(eigvals, nfilling, nk*nk)
        print("efield, efermi:", efield, efermi)
        # save the eigvals and eigvecs to the file
        f.create_dataset(f"e{efield}/eigvals", data=eigvals-efermi)
        # f.create_dataset(f"e{efield}/eigvecs", data=eigvecs)
        f.create_dataset(f"e{efield}/vbm", data=vbm-efermi)
        f.create_dataset(f"e{efield}/cbm", data=cbm-efermi)
        f.create_dataset(f"e{efield}/efermi", data=efermi)


efield, efermi: -0.5 -1.2108392009824005
efield, efermi: -0.35 -0.13527981027828917
efield, efermi: -0.3 0.22463188082338306
efield, efermi: -0.2 0.9461977974906194
efield, efermi: -0.15 1.3077151476500841
efield, efermi: -0.1 1.669630781311645
efield, efermi: -0.05 2.0318830226347506
efield, efermi: 0.0 2.3945329176082515
efield, efermi: 0.05 2.7441479834537024
efield, efermi: 0.1 3.093929302906691
efield, efermi: 0.15 3.444042827302647
efield, efermi: 0.2 3.794554643497816
efield, efermi: 0.3 4.497052037735717
efield, efermi: 0.35 4.849175356356369
efield, efermi: 0.5 5.9097297738682695
